In [4]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
%precision 4

import pyjags
import numpy as np

def summary(samples, varname, p=95, burnin=0, thin=1):
    values = samples[varname]
    (nb, iter, chains) = values.shape
    
    for i in range(nb):
        data = values[i][burnin::thin]

        ci = np.percentile(data, [100-p, p])
        
        print('{:<8}[{}] mean = {:>12.4f} (std = {:>10.4f}), {}% credible interval [{:>5.2f} {:>5.2f}]'.format(
          varname, i, np.mean(data), np.std(data), p, *ci))

In [20]:
sample = np.array([
        [10, 5, 12, 14, 30],
        [20, 14, 18, 22, 50],
        [20, 14, 18, 15, 25],
        [3, 2, 1, 2, 0]
    ])

cp, T = sample.shape

print(cp, T)

code = """
# bugs model for poisson distribs

model{
        for (i in 1:cp) {
            for (t in 1:T) {
                n[i, t] ~ dpois(a[i] * N[t])
            }
        }
        
        for (t in 1:T) {
            N[t]   ~ dpois(NN)
        }
        NN      ~ dunif(10,100)
        
        for (i in 1:cp) {
            b[i]   ~ dnorm(0, 10) I(0,)
        }
        
        for (i in 1:cp) {
            a[i]  <- b[i] / sum(b)
        }
}
"""

myvars=['NN', 'N', 'a']

%time model = pyjags.Model(code, data=dict(n=sample, cp=cp, T=T), chains=4, threads=4)
%time samples = model.sample(20000, vars=myvars)
samples.keys()

4 5
adapting: iterations 4000 of 4000, elapsed 0:00:00, remaining 0:00:00
CPU times: user 480 ms, sys: 0 ns, total: 480 ms
Wall time: 154 ms
sampling: iterations 22851 of 80000, elapsed 0:00:01, remaining 0:00:02
sampling: iterations 55332 of 80000, elapsed 0:00:01, remaining 0:00:01
sampling: iterations 75611 of 80000, elapsed 0:00:02, remaining 0:00:00
sampling: iterations 80000 of 80000, elapsed 0:00:02, remaining 0:00:00
CPU times: user 6.62 s, sys: 0 ns, total: 6.62 s
Wall time: 2.01 s


dict_keys(['N', 'a', 'NN'])

In [21]:
burnin = 5000
        
for varname in myvars:
    summary(samples, varname, burnin=burnin, thin=4)

NN      [0] mean =      58.5320 (std =     4.8235), 95% credible interval [50.78 66.55]
N       [0] mean =      55.8446 (std =     5.8288), 95% credible interval [47.00 66.00]
N       [1] mean =      46.1507 (std =     5.6240), 95% credible interval [37.00 56.00]
N       [2] mean =      53.7283 (std =     5.7567), 95% credible interval [45.00 64.00]
N       [3] mean =      55.8379 (std =     5.8477), 95% credible interval [46.00 66.00]
N       [4] mean =      80.0961 (std =     6.5587), 95% credible interval [69.00 91.00]
a       [0] mean =       0.2415 (std =     0.0244), 95% credible interval [ 0.20  0.28]
a       [1] mean =       0.4169 (std =     0.0283), 95% credible interval [ 0.37  0.46]
a       [2] mean =       0.3112 (std =     0.0266), 95% credible interval [ 0.27  0.36]
a       [3] mean =       0.0305 (std =     0.0101), 95% credible interval [ 0.02  0.05]


## Now with some noise

In [87]:
sample = np.array([
        [10, 5, 22, 14, 130],
        [20, 14, 18, 22, 50],
        [20, 600, 18, 200, 25],
        [3, 2, 1, 7, 0],
        [500, 400, 1000, 300, 300]
    ])

cp, T = sample.shape

print(cp, T)

code = """
# bugs model for poisson distribs

model{
        for (i in 1:cp) {
            for (t in 1:T) {
                m[i, t]       ~ dpois(a[i] * N[t])
                noise[i, t]   ~ dbern(p_noise)
                outl[i, t]    ~ dnorm(0, tau_outl)
                mm[i, t, 1]  <- m[i, t]
                mm[i, t, 2]  <- m[i, t] * (3 + outl[i, t] * outl[i, t])
                
                n[i, t]       ~ dnorm(mm[i, t, choice[i, t]], tau)
                choice[i, t] <- noise[i, t] + 1
            }
        }
        
        tau         <- 1 / sigma / sigma
        sigma        ~ dunif(0.0001, 0.001)
        
        tau_outl    <- 1 / sigma_outl / sigma_outl
        sigma_outl   ~ dunif(0.1, 5)
        
        
        for (t in 1:T) {
            N[t]   ~ dpois(NNN)
        }
        
        NNN      ~ dunif(NN/2, NN*2)
        p_noise  ~ dbeta(10, 1000)
        
        for (i in 1:cp) {
            b[i]   ~ dunif(0, 1)
        }
        
        for (i in 1:cp) {
            a[i]  <- b[i] / sum(b)
        }
}
"""

myvars=['NN', 'N', 'a', 'p_noise', 'noise', 'sigma', 'sigma_outl']

%time model = pyjags.Model(code,  \
                           data=dict(n=sample, cp=cp, T=T, NN=sample.sum(axis=0).mean()),  \
                           chains=4, threads=4, adapt=4000)
%time samples = model.sample(20000, vars=myvars)
samples.keys()

5 5
adapting: iterations 4348 of 16000, elapsed 0:00:01, remaining 0:00:02
adapting: iterations 11956 of 16000, elapsed 0:00:01, remaining 0:00:00
adapting: iterations 15036 of 16000, elapsed 0:00:02, remaining 0:00:00
adapting: iterations 16000 of 16000, elapsed 0:00:02, remaining 0:00:00
CPU times: user 6.06 s, sys: 0 ns, total: 6.06 s
Wall time: 2 s
sampling: iterations 6125 of 80000, elapsed 0:00:01, remaining 0:00:10
sampling: iterations 13767 of 80000, elapsed 0:00:01, remaining 0:00:06
sampling: iterations 17713 of 80000, elapsed 0:00:02, remaining 0:00:06
sampling: iterations 25369 of 80000, elapsed 0:00:02, remaining 0:00:05
sampling: iterations 32193 of 80000, elapsed 0:00:03, remaining 0:00:04
sampling: iterations 40171 of 80000, elapsed 0:00:03, remaining 0:00:03
sampling: iterations 49186 of 80000, elapsed 0:00:04, remaining 0:00:03
sampling: iterations 57166 of 80000, elapsed 0:00:05, remaining 0:00:02
sampling: iterations 63538 of 80000, elapsed 0:00:05, remaining 0:00:0

dict_keys(['noise', 'NN', 'sigma', 'p_noise', 'N', 'a', 'sigma_outl'])

In [88]:
for varname in myvars:
    if varname != 'noise':
        summary(samples, varname, burnin=0, thin=4)

NN      [0] mean =     736.2000 (std =     0.0000), 95% credible interval [736.20 736.20]
N       [0] mean =     409.4844 (std =   134.6472), 95% credible interval [281.00 654.00]
N       [1] mean =     781.3936 (std =    64.8197), 95% credible interval [693.00 886.00]
N       [2] mean =     625.3432 (std =   233.7227), 95% credible interval [339.00 902.00]
N       [3] mean =     557.0915 (std =    53.7065), 95% credible interval [487.00 645.00]
N       [4] mean =     522.8370 (std =    60.4425), 95% credible interval [448.00 627.00]
a       [0] mean =       0.0621 (std =     0.0116), 95% credible interval [ 0.05  0.08]
a       [1] mean =       0.0247 (std =     0.0064), 95% credible interval [ 0.02  0.04]
a       [2] mean =       0.2941 (std =     0.0461), 95% credible interval [ 0.23  0.36]
a       [3] mean =       0.0041 (std =     0.0019), 95% credible interval [ 0.00  0.01]
a       [4] mean =       0.6150 (std =     0.0556), 95% credible interval [ 0.54  0.69]
p_noise [0] mean =  

In [90]:
noise = samples["noise"]
(a, b, d, e) = noise.shape

noise.mean(axis=2).mean(axis=2)

array([[ 0.25  ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.25  ,  0.25  ,  0.5   ,  0.25  ,  0.75  ],
       [ 0.25  ,  0.    ,  0.    ,  0.    ,  0.25  ],
       [ 0.    ,  0.    ,  0.    ,  0.5   ,  0.0143],
       [ 0.75  ,  0.    ,  0.5   ,  0.    ,  0.    ]])